# Estimating GGM using Score Matching

In [1]:
using HD

Generate Gaussian data from which we will be learning the structure. The covariance is a Toeplitz matrix, that is, $\Sigma = (\sigma_{ab})$ with $\sigma_{ab} = \rho^{|a-b|}$.

In [53]:
p = 10
n = 100

Sigma = zeros(Float64, p, p)
rho = 0.8
for a=1:p
    for b=1:p
        Sigma[a,b] = rho^abs(a-b)
    end
end
Theta = inv(Sigma)

X = randn(n, p) * sqrtm(Sigma);

Constructing for each $i\in[n]$ a matrix $D(x_i)$

In [3]:
# returns sparse matrix of size p x (p*K + p * (p-1) / 2 * L)
function getD(
    X::Array{Float64, 2}, rowInd::Int64, K::Int64, L::Int64,
    phi_node_d::Function, phi_edge_d::Function)
    
    
    n, p = size(X)
    # will be used to create a sparse matrix at the end
    rI = Array(Int64, 0)
    cI = Array(Int64, 0)
    V = Array(Float64, 0)
    
    # process node elements
    for a=1:p
        _ri = a
        for k=1:K
            _ci = (a-1)*K+k
            _v = phi_node_d(X[rowInd, a], k)
            
            push!(rI, _ri)
            push!(cI, _ci)
            push!(V, _v)
        end
    end
    
    numEdges = 0
    # process edges
    for a=1:p
        for b=a+1:p
            numEdges += 1
            
            # derivative with respect to first argument
            _ri = a
            for l=1:L
                _ci = p*K + (numEdges-1)*L + l                
                _v = phi_edge_d(X[rowInd, a], X[rowInd, b], l, 1)
                
                push!(rI, _ri)
                push!(cI, _ci)
                push!(V, _v)
            end
            
            # derivative with respect to second argument
            _ri = b
            for l=1:L
                _ci = p*K + (numEdges-1)*L + l                
                _v = phi_edge_d(X[rowInd, a], X[rowInd, b], l, 2)
                
                push!(rI, _ri)
                push!(cI, _ci)
                push!(V, _v)
            end          
            
        end
    end
    
    sparse(rI, cI, V, p, int(p*K+p*(p-1)/2*L))    
end

getD (generic function with 1 method)

Constructing matrix $E$

In [4]:
function getE(
    X::Array{Float64, 2}, K::Int64, L::Int64,
    phi_node_d_2::Function, phi_edge_d_2::Function)
    
    n, p = size(X)
    E = zeros(Float64, int(p*K+p*(p-1)/2*L), 1)
    for rowInd=1:n
        # process node elements
        for a=1:p        
            for k=1:K
                _ci = (a-1)*K+k
                E[_ci] = E[_ci] + phi_node_d_2(X[rowInd, a], k)                        
            end
        end    
        
        
        # process edges
        numEdges = 0        
        for a=1:p
            for b=a+1:p
                numEdges += 1                                        
                for l=1:L
                    _ci = p*K + (numEdges-1)*L + l                
                    E[_ci] = E[_ci] + phi_edge_d_2(X[rowInd, a], X[rowInd, b], l)
                end            
            end
        end
    end
    E
end

getE (generic function with 1 method)

Constructing node and edge handles

In [5]:
function phi_node(x::Float64, k::Int64)
    k == 1 ? - x^2 / 2. : 0.
end;

In [6]:
function phi_edge(x::Float64, y::Float64, k::Int64)
    k == 1 ? - x * y : 0.
end;

In [7]:
function phi_edge_der(x::Float64, y::Float64, k::Int64, whichArgument::Int64)
    if k == 1
        if whichArgument == 1
            return -y
        elseif whichArgument == 2
            return -x
        else
            throw(ArgumentError("whichArgument should be either 1 or 2"))
        end
    else
        return 0.
    end    
end;

In [8]:
function phi_node_der(x::Float64, k::Int64)
    k == 1 ? - x : 0.
end;

In [9]:
function phi_edge_der_2(x::Float64, y::Float64, k::Int64)
    return 0.
end;

In [10]:
function phi_node_der_2(x::Float64, k::Int64)
    k == 1 ? -1. : 0.
end;

## Run Group Lasso

In [11]:
K = 1
L = 1
numFeatures = int(p*K+p*(p-1)/2*L)
groups = Array(Array{Int64, 1}, numFeatures)
for i=1:numFeatures
    groups[i] = [i]
end
mask = ones(Float64, numFeatures)
mask[1:p*K] = 0.;

In [47]:
DD = zeros(Float64, numFeatures, numFeatures)
@time for i=1:n
    D = getD(X, i, K, L, phi_node_der, phi_edge_der)
    broadcast!(+, DD, DD, D'D)
end

elapsed time: 0.006776446 seconds (8935224 bytes allocated)


In [48]:
E = getE(X, K, L, phi_node_der_2, phi_edge_der_2);

In [54]:
eTmp = - DD \ E;
eTheta = zeros(Float64, p, p)

for a=1:p        
    for k=1:K
        _ci = (a-1)*K+k
        eTheta[a,a] = eTmp[_ci]
    end
end            
        
numEdges = 0        
for a=1:p
    for b=a+1:p
        numEdges += 1                                        
        for l=1:L
            _ci = p*K + (numEdges-1)*L + l                
            eTheta[a,b] = eTmp[_ci]
            eTheta[b,a] = eTmp[_ci]
        end            
    end
end

In [55]:
eTheta, Theta

(
10x10 Array{Float64,2}:
  3.69465    -2.61944   -0.81703    …   0.112437   -0.191774   -0.0269485
 -2.61944     4.40339   -1.25266        0.10087     0.065415   -0.07639  
 -0.81703    -1.25266    4.44752       -0.0449493  -0.0801923   0.574739 
  0.773916   -0.237338  -2.50784       -0.153403   -0.295028    0.166991 
 -0.552158    0.258944   0.711903      -0.266807   -0.108105    0.529375 
  0.211166   -0.492483  -0.203214   …   0.0611513   0.121856   -0.544949 
 -0.543465    0.587717  -0.146314      -2.04211     0.149969   -0.410102 
  0.112437    0.10087   -0.0449493      4.18931    -1.68506    -0.11371  
 -0.191774    0.065415  -0.0801923     -1.68506     4.09715    -2.35692  
 -0.0269485  -0.07639    0.574739      -0.11371    -2.35692     3.28987  ,

10x10 Array{Float64,2}:
  2.77778      -2.22222      -5.05768e-16  …  -3.11479e-16   1.38778e-16
 -2.22222       4.55556      -2.22222          6.16791e-16  -3.08395e-16
  4.14483e-18  -2.22222       4.55556         -7.46317e-16   4